In [67]:
from pathlib import Path
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
import requests

import time
from datetime import datetime
# 加载.env文件
load_dotenv("config.env")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_content(word):
    print(f"\n==== 开始为单词 '{word}' 生成内容 ====")
    print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    start_time = time.time()
    prompt = """你的角色是英语单词学习助手，使用者是小学生。会发给你不熟悉的单词，需要你来讲解，我会把你的讲解生成音频播放出来，有两个要求：
    1.严格按照我发的例子，来生成讲解。
    2.只要是讲解单词就必须生成音频、写入数据库
    
    范例：
    '''
    Hello kids! 今天我们要学习一个单词："apple" (苹果)
    
    1. **How to Read (怎么读)**: 我们把它分成两个部分，或者说音节："ap - ple."
      - 第一个音节是 "ap"，听起来像"cat" (猫) 里的"a"，然后加上"p"。
      - 第二个音节是 "ple"，听起来像 "pull" (拉)，但是在最后加上一个"l"。
      - 现在把它们连起来： "apple"。跟我一起重复："apple, apple, apple。" 太棒了！
    
    2. **What it Means (什么意思)**: Apple 是一种水果，长在树上。它可以是红色、绿色或黄色的。苹果很脆也很甜！
    
    3. **Sentence Example (例句)**: 我们用 "apple" 造个句子："I have an apple in my lunchbox." (我的午餐盒里有一个苹果。)
    
    记住，苹果不仅好吃，而且很健康！继续练习，你很快就会成为一个苹果小专家
    '''
    
    返回结果已稳定的json输出：
    {
    "type": "词性(英文)",
    "meaning": "词意(中文)",
    "sentence_example":"对应生成的范例 Sentence Example"
    "speech": "对应生成的范例内容"
    }"""
    # 定义function call配置
    function_call = [
        {
           "type": "function",
           "function": {
               "name": "generate_audio",
               "description": "根据内容生成音频",
               "parameters": {
                   "type": "object",
                   "required": [
                       "content",
                       "word"
                   ],
                   "properties": {
                       "content": {
                           "type": "string",
                           "description": "要转换为音频的文本内容"
                       },
                       "word": {
                           "type": "string",
                           "description": "内容中讲解的单词"
                       }
                   },
                   "additionalProperties": False
               },
               "strict": True
           }
        },
        {
          "type": "function",
          "function": {
            "name": "insert_db_content",
            "description": "Inserts content with a given word and its tips into the database",
            "parameters": {
              "type": "object",
              "required": [
                "act_type",
                "word",
                "tips"
              ],
              "properties": {
                "act_type": {
                  "type": "string",
                  "description": "Action type, must be 'add'",
                  "enum": [
                    "add"
                  ]
                },
                "word": {
                  "type": "string",
                  "description": "The word to be added"
                },
                "tips": {
                  "type": "string",
                  "description": "Part of speech and meaning of the word"
                }
              },
              "additionalProperties": False
            },
            "strict": True
          }
        }
    ]
    
    response = client.chat.completions.create(
       model="gpt-4o",
       messages=[
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": prompt
                   }
               ]
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "text",
                       "text": f"单词：{word}"
                   }
               ]
           }
       ],
       response_format={
           "type": "json_object"
       },
       tools=function_call,
       temperature=1,
       max_tokens=4465,
       top_p=1,
       frequency_penalty=0,
       presence_penalty=0
   )
    
    # 记录API调用耗时
    api_time = round(time.time() - start_time, 2)
    print(f"\nAPI调用完成，耗时: {api_time}秒")
    
    content = response.choices[0].message.content
    if not content:
        print("\n\nopenai 返回异常：\n")
        print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))
        return
    print("\n=== 生成的教学内容 ===")
    print(json.dumps(json.loads(content), indent=2, ensure_ascii=False))
    
    tool_calls = response.choices[0].message.tool_calls
    print("\n=== 开始处理函数调用 ===")
    function_result = handle_tool_calls(tool_calls)
    
    total_time = round(time.time() - start_time, 2)
    print(f"\n==== 处理完成 ====")
    print(f"总耗时: {total_time}秒")
    print(f"结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\n\n返回的原始结果：\n")
    print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))
    
    return function_result

def generate_audio(content: str, word: str):
    """
    生成音频的具体实现函数
    Args:
        content: 要转换为音频的文本内容
        word: 内容中讲解的单词
    """
    print(f"\n=== 开始生成音频 ===")
    print(f"单词: {word}")
    print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if not content or not word:
        print("错误: 生成语音内容不能为空")
        return
    
    # 确保目录存在
    speech_file_path = f"/Users/ycyang/code/test/data/audio/{word}.mp3"
    os.makedirs(os.path.dirname(speech_file_path), exist_ok=True)
    
    start_time = time.time()
    try:
        print("调用 OpenAI TTS API...")
        response = client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=content
        )
        
        # 直接获取响应内容并写入文件
        print(f"正在保存音频文件...")
        with open(speech_file_path, "wb") as file:
            for chunk in response.iter_bytes():
                file.write(chunk)
        
        file_size = Path(speech_file_path).stat().st_size / 1024  # KB
        print(f"音频文件生成完成")
        print(f"保存位置: {speech_file_path}")
        print(f"文件大小: {file_size:.2f} KB")
        
        total_time = round(time.time() - start_time, 2)
        print(f"\n音频生成成功")
        print(f"总耗时: {total_time}秒")
        
    except Exception as e:
        print(f"错误: 生成音频失败")
        print(f"错误信息: {str(e)}")
        print(f"时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        raise

def handle_tool_calls(tool_calls):
    if not tool_calls:
        print("没有函数调用需要处理")
        return {}
    
    print(f"处理时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"找到 {len(tool_calls)} 个函数调用")
    
    result = {}
    for index, call in enumerate(tool_calls, 1):
        func_name = call.function.name
        func_args = json.loads(call.function.arguments)
        
        print(f"\n=== 处理第 {index} 个函数调用 ===")
        print(f"函数名称: {func_name}")
        print(f"参数详情:")
        print(json.dumps(func_args, indent=2, ensure_ascii=False))
        
        # 检查并执行函数
        if func_name in function_mapping:
            start_time = time.time()
            try:
                print(f"\n开始执行函数: {func_name}")
                function_mapping[func_name](**func_args)
                
                execution_time = round(time.time() - start_time, 2)
                print(f"函数执行成功")
                print(f"执行耗时: {execution_time}秒")
                
                result[func_name] = {
                    "status": "success",
                    "args": func_args,
                    "execution_time": execution_time,
                    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
            except Exception as e:
                execution_time = round(time.time() - start_time, 2)
                print(f"函数执行失败")
                print(f"错误信息: {str(e)}")
                print(f"执行耗时: {execution_time}秒")
                
                result[func_name] = {
                    "status": "error",
                    "error": str(e),
                    "args": func_args,
                    "execution_time": execution_time,
                    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
        else:
            print(f"警告: 未知的函数 {func_name}")
            result[func_name] = {
                "status": "unknown_function",
                "args": func_args,
                "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
    
    return result

def insert_db_content(act_type: str, word: str, tips: str) -> bool:
   """
   向数据库插入单词内容
   
   Args:
       word: 单词
       tips: 提示信息/注释
   
   Returns:
       bool: 插入是否成功
   """
   print(f"\n=== 开始插入数据库 ===")
   print(f"单词: {word}")
   print(f"提示: {tips}")
   print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
   
   url = "http://localhost:8111/game.php"
   
   # 准备请求数据
   payload = {
       "act_type": act_type,
       "word": word,
       "tips": tips
   }
   
   headers = {
       "Content-Type": "application/json"
   }
   
   try:
       print("发送请求...")
       response = requests.post(
           url=url,
           data=json.dumps(payload),
           headers=headers
       )
       
       print(f"状态码: {response.status_code}")
       
       if response.status_code == 200:
           data = response.json()
           if data.get("resCode") == 200:
               print("数据插入成功")
               return True
           else:
               print(f"插入失败: {data.get('message', '未知错误')}")
               return False
       else:
           print(f"请求失败: HTTP {response.status_code}")
           return False
           
   except Exception as e:
       print(f"错误: 请求失败")
       print(f"错误信息: {str(e)}")
       print(f"时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
       return False
   finally:
       print(f"=== 插入操作结束 ===")

# 定义函数映射字典
function_mapping = {
    "generate_audio": generate_audio,
    "insert_db_content": insert_db_content,
}



In [71]:
generate_content("percent")


==== 开始为单词 'percent' 生成内容 ====
开始时间: 2024-11-26 18:56:07

API调用完成，耗时: 21.93秒

=== 生成的教学内容 ===
{
  "type": "noun",
  "meaning": "百分比",
  "sentence_example": "When you save money in a bank, they usually give you some percent of interest each year. (当你在银行存钱时，他们通常每年给你一定的利息百分比。)",
  "speech": "Hello kids! 今天我们要学习一个单词：\"percent\" (百分比)\n\n1. **How to Read (怎么读)**: 我们把它分成两个部分，或者说音节：\"per - cent.\"\n  - 第一个音节是 \"per\"，听起来像\"purr\" (猫叫) 的声音。\n  - 第二个音节是 \"cent\"，听起来像说 \"send\" (发送) 的时候，去掉最后的\"d\"。\n  - 现在把它们连起来： \"percent\"。跟我一起重复：\"percent, percent, percent。\" 太棒了！\n\n2. **What it Means (什么意思)**: Percent 是用来描述一个数占整体的百分之多少。比如说，50% 就是整体的一半。\n\n3. **Sentence Example (例句)**: 我们用 \"percent\" 造个句子：\"When you save money in a bank, they usually give you some percent of interest each year.\" (当你在银行存钱时，他们通常每年给你一定的利息百分比。)\n\n记住，了解百分比可以帮助你更好地理解数字和百分比之间的关系！"
}

=== 开始处理函数调用 ===
处理时间: 2024-11-26 18:56:29
找到 2 个函数调用

=== 处理第 1 个函数调用 ===
函数名称: generate_audio
参数详情:
{
  "content": "Hello kids! 今天我们要学习一个单词：\"pe

{'generate_audio': {'status': 'success',
  'args': {'content': 'Hello kids! 今天我们要学习一个单词："percent" (百分比)\n\n1. **How to Read (怎么读)**: 我们把它分成两个部分，或者说音节："per - cent."\n  - 第一个音节是 "per"，听起来像"purr" (猫叫) 的声音。\n  - 第二个音节是 "cent"，听起来像说 "send" (发送) 的时候，去掉最后的"d"。\n  - 现在把它们连起来： "percent"。跟我一起重复："percent, percent, percent。" 太棒了！\n\n2. **What it Means (什么意思)**: Percent 是用来描述一个数占整体的百分之多少。比如说，50% 就是整体的一半。\n\n3. **Sentence Example (例句)**: 我们用 "percent" 造个句子："When you save money in a bank, they usually give you some percent of interest each year." (当你在银行存钱时，他们通常每年给你一定的利息百分比。)\n\n记住，了解百分比可以帮助你更好地理解数字和百分比之间的关系！',
   'word': 'percent'},
  'execution_time': 10.32,
  'timestamp': '2024-11-26 18:56:39'},
 'insert_db_content': {'status': 'success',
  'args': {'act_type': 'add', 'word': 'percent', 'tips': 'noun: 百分比'},
  'execution_time': 0.43,
  'timestamp': '2024-11-26 18:56:40'}}